## Create Base Portfolios

This notebook creates Base Portfolios in Moody's Risk Modeler

### Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch, validate_batch
from helpers.database import execute_query
from helpers.step import get_last_step_run
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

# Flag to track validation state - allows notebook to complete gracefully on failure
validation_failed = False
validation_errors = []

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_02_Create_Base_Portfolios.ipynb')

# Display context
ux.header("Create Base Portfolios")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

### Retrieve Base Portfolio Batch

In [ ]:
# Retrieve Portfolio Creation batch from Stage_01/Step_03
ux.subheader("Retrieve Base Portfolio Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.PORTFOLIO_CREATION not in batches:
    raise ValueError(f"Portfolio Creation batch not found. Available: {list(batches.keys())}")

portfolio_batch_id = int(batches[BatchType.PORTFOLIO_CREATION])

ux.success(f"✓ Retrieved Portfolio Creation batch: ID={portfolio_batch_id}")
step.log(f"Retrieved Portfolio Creation batch: ID={portfolio_batch_id}")

### Review Base Portfolio Batch Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(portfolio_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch
jobs = get_batch_jobs(portfolio_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Show sample configurations
if job_count > 0:
    ux.info("\nSample EDM Configurations (first 5):")
    for i, job in enumerate(jobs[:5]):
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        if not config_result.empty:
            config = config_result.iloc[0]['job_configuration_data']
            ux.info(f"  {i+1}. {config.get('Portfolio', 'N/A'), config.get('Database', 'N/A')}")

# Validate batch before submission
ux.subheader("Validate Batch")
validation_errors = validate_batch(portfolio_batch_id)

if validation_errors:
    validation_failed = True
    ux.error("✗ Batch validation failed:")
    for error in validation_errors:
        ux.error(f"  {error}")
    step.log(f"Batch validation failed: {len(validation_errors)} error(s)")
else:
    ux.success("✓ Batch validation passed")
    step.log(f"Verified batch: {job_count} jobs ready for submission")

### Submit Base Portfolio Batch to Moody's

In [ ]:
# Submit batch to Moody's API
if validation_failed:
    ux.warning("⏭ Skipping submission due to validation failure")
    result = None
    failed_count = 0
else:
    ux.subheader("Submit Batch to Moody's")

    ux.info("")
    ux.info("Submission Process:")
    ux.info("  • Each job will create a Portfolio in Moody's")
    ux.info("  • Jobs will transition to SUBMITTED status")
    ux.info("  • Batch will transition to ACTIVE status")
    ux.info("")

    # Submit
    ux.info("\nSubmitting batch...")

    # Pass step.step_id to associate batch with this step (not the creation step)
    result = submit_batch(portfolio_batch_id, IRPClient(), step_id=step.step_id)

    # Display results
    ux.success(f"\n✓ Batch submission completed")
    ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
    ux.info(f"  Status: {result['batch_status']}")

    # Check for errors
    failed_count = len([j for j in result['jobs'] if 'error' in j])
    if failed_count > 0:
        ux.warning(f"\n⚠ {failed_count} job(s) failed to submit")
        for job_result in result['jobs']:
            if 'error' in job_result:
                ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

    step.log(f"Batch submitted: {result['submitted_jobs']} jobs, {failed_count} failed")

### Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

if validation_failed:
    # Handle validation failure
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    from helpers.teams_notification import send_validation_failure_notification
    
    error_message = "\n".join(validation_errors)
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    # Send Teams notification
    send_validation_failure_notification(
        cycle_name=context.cycle_name,
        stage_name=context.stage_name,
        step_name=context.step_name,
        validation_errors=validation_errors,
        notebook_path=str(context.notebook_path)
    )
    
    ux.error("\n" + "="*60)
    ux.error("BATCH VALIDATION FAILED")
    ux.error("="*60)
    ux.info(f"\nBatch ID: {portfolio_batch_id}")
    ux.error(f"\nValidation errors ({len(validation_errors)}):")
    for error in validation_errors:
        ux.error(f"  {error}")
    ux.info("\nPlease fix the validation errors and retry.")

elif failed_count > 0:
    # Handle submission failures
    failed_job_errors = [
        f"Job {j['job_id']}: {j['error']}" 
        for j in result['jobs'] if 'error' in j
    ]
    error_message = f"{failed_count} job(s) failed to submit:\n" + "\n".join(failed_job_errors)
    
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    ux.error("\n" + "="*60)
    ux.error("BATCH SUBMISSION FAILED")
    ux.error("="*60)
    ux.info(f"\nBatch ID: {portfolio_batch_id}")
    ux.info(f"Submitted: {result['submitted_jobs']} job(s)")
    ux.error(f"Failed: {failed_count} job(s)")
    ux.info("\nFailed jobs:")
    for error in failed_job_errors:
        ux.error(f"  {error}")
    ux.info("\nPlease review the errors and resubmit failed jobs.")

else:
    # Complete the step successfully
    output_data = {
        'batch_id': portfolio_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': result['batch_status'],
        'submitted_jobs': result['submitted_jobs'],
        'failed_jobs': failed_count
    }
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("BASE PORTFOLIO BATCH SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nSubmitted {result['submitted_jobs']} job(s) to Moody's API")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNote: Portfolio Creation is synchronous - all jobs complete immediately")
    ux.info("Next: Proceed to MRI Import or other data loading steps")